https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/hela-sverige-ska-fungera--kristdemokraternas_H9024221

In [5]:
import pandas as pd
import requests

In [6]:
# currency multiplier used in this budget
multiplier = 1_000

In [7]:
search_url = 'https://data.riksdagen.se/dokumentlista/'
params = {
    'sok': 'utgiftsområde',
    'doktyp': 'mot',
    'rm': '2021/22',
    'parti': 'KD',
    'sort': 'rel',
    'sortorder': 'desc',
    'utformat': 'json',
    'a': 's'}

resp = requests.get(search_url, params=params)
data = resp.json()
docs = pd.DataFrame(data['dokumentlista']['dokument'])
docs = docs[['titel', 'undertitel', 'rm', 'dokument_url_html']]

In [8]:
docs['Utgiftsområde'] = docs.titel.str.extract('Utgiftsområde (\d+)')
docs['Utgiftsområde'] = docs['Utgiftsområde'].astype(int)
docs = docs.sort_values('Utgiftsområde').reset_index(drop=True)

In [9]:
def find_matching_table(tables):
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    for table in tables:
        if table.columns.shape == (4,) and 'Avvikelse från regeringen' in table.columns:
            break
    return table

In [10]:
def fetch_table(url, area):
    url = f'http:{url}'
    tables = pd.read_html(url, encoding='utf8', header=0)
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    df = find_matching_table(tables)
    df.columns = cols
    df = df.dropna(how='all')
    df = df[~df.Anslag.str.startswith('Summa')]
    df['Utgiftsområde'] = area
    return df

In [11]:
tables = []

for _, row in docs.iterrows():
    try:
        table = fetch_table(row.dokument_url_html, area=row.Utgiftsområde)
        tables.append(table)
    except ValueError:  # missing images...
        continue

In [18]:
df = pd.concat(tables, sort=False)

In [19]:
df = df.dropna(how='all')

In [20]:
df = df.drop('Regeringens förslag', axis=1)

In [21]:
df = df.replace('±0', '0')
df = df.rename(columns={'Avvikelse från regeringen': '2022'})
df['2022'] = df['2022'].str.replace('\s+', '', regex=True)
df['2022'] = df['2022'].str.replace('−', '-')

In [22]:
df['2022'] = df['2022'].astype(int) * multiplier

In [26]:
df.to_csv('../data/budgetmotion-2022-kd.csv', index=False)